<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/potato_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Model to detect potato diseases

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
potato_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/Colab Notebooks/Computer Vision/potato disease/train',
                                                    labels='inferred',
                                                    label_mode='int',
                                                    batch_size=32,
                                                    image_size=(256, 256),
                                                    shuffle=True,
                                                    validation_split=0.1,
                                                    subset="training",
                                                    seed=55)

Found 2152 files belonging to 3 classes.
Using 1937 files for training.


In [13]:
potato_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [19]:
classes = potato_dataset.class_names
classes

['Early_blight', 'Late_blight', 'healthy']

In [17]:
for images, labels in potato_dataset.take(1):
  print(images.shape)
  print(labels)

(32, 256, 256, 3)
tf.Tensor([0 1 0 1 1 1 1 0 0 2 1 1 0 1 2 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1], shape=(32,), dtype=int32)


In [ ]:
potato_dataset.